In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
from scipy import stats
import plotly.express as px
import toml
import psrc_theme

%matplotlib inline
from IPython.display import display, HTML

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

input_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'input_configuration.toml'))
valid_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))

# create connection
conn = create_engine('sqlite:///../../../../inputs/db/'+input_config['db_name'])

# summary data location
csv_path = os.path.join(valid_config['model_dir'], 'outputs/validation')

# table format
pd.options.display.float_format = '{:0,.2f}'.format
format_percent = "{:,.2%}".format

# Link Expansion Analysis
Testing 2023 base year with Lynnwood extension and 2-Line connection to Seattle

### Line Level Totals


In [2]:
df_obs = pd.read_sql(
    "SELECT * FROM light_rail_station_boardings WHERE year IN (2023, 2025)",
    con=conn,
)
df_obs.rename(columns={"boardings": "observed_boardings"}, inplace=True)

df_obs
df_obs_pivot = pd.pivot_table(df_obs, values='observed_boardings', columns='year', index='station_name', aggfunc=np.sum)
df_obs_pivot = df_obs_pivot.fillna(0)
df_obs_pivot = df_obs_pivot.reset_index()
_df = df_obs.loc[df_obs['year']==2025,['station_name','emme_node']]
df_obs_pivot = df_obs_pivot.merge(_df, on=['station_name'])

df = pd.read_csv(os.path.join(valid_config['model_dir'], 'outputs/transit/boardings_by_stop.csv'))
df = df[df["i_node"].isin(df_obs["emme_node"])]
df = df.merge(df_obs_pivot, left_on="i_node", right_on="emme_node")
df.rename(columns={"total_boardings": "Model Boardings"}, inplace=True)
df.index = df["station_name"]
df_total = df.copy()[["Model Boardings",2023,2025, ]]
df_total.loc["Total", ["Model Boardings", 2023,2025]] = (
    df[["Model Boardings", 2023,2025]].sum().values
)
df_total
df_total.rename(columns={2023: 'Observed Spring 2023 (Pre-Lynnwood)', 2025: 'Observed Fall 2024 (Post-Lynnwood)' }, 
               inplace=True)
line_1_stations = ["Lynnwood Station","Mountlake Terrace Station","185th St Station","145th St Station",
                "Northgate Station",'Roosevelt Station','University District Station','Husky Stadium',
              'Capitol Hill','Westlake','Symphony',
              "Pioneer Square","Int'l District",
                 'Stadium','SODO','Beacon Hill','Mount Baker','Columbia City','Othello','Rainier Beach',
                "Tukwila Int'l Blvd",'SeaTac/Airport','Angle Lake']
line_1_stations = [x for x in line_1_stations if x in df_total.index]
df_1_line = df_total.loc[line_1_stations]

line_2_stations = ['South Bellevue Station','East Main Station','Bellevue Station','Wilburton Station',
             'Spring District/120th Station','Bel-Red/130th Station','Overlake Village Station',
             'Redmond Technology Station']
line_2_stations = [x for x in line_2_stations if x in df_total.index]
df_2_line = df_total.loc[line_2_stations]


C:\Users\modeller\AppData\Local\Temp\ipykernel_22560\2648845712.py:8: FutureWarning:

The provided callable <function sum at 0x0000024E426ED1C0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



In [3]:
df_total

,Model Boardings,Observed Spring 2023 (Pre-Lynnwood),Observed Fall 2024 (Post-Lynnwood)
station_name,,,
Husky Stadium,"3,919.56","6,134.00","5,122.00"
Westlake,"11,302.09","8,845.00","10,737.00"
Symphony,"6,422.60","4,324.00","4,783.00"
Pioneer Square,"5,205.49","2,400.00","3,008.00"
Int'l District,"1,839.46","4,399.00","5,372.00"
Capitol Hill,"8,174.24","7,667.00","8,207.00"
Stadium,522.98,"1,731.00","1,813.00"
SODO,"3,311.34","1,669.00","1,899.00"
Beacon Hill,"1,444.32","2,147.00","2,672.00"


In [4]:
df_total.loc[df_total.index.isin(line_1_stations),'Line'] = '1-Line'
df_total.loc[df_total.index.isin(line_2_stations),'Line'] = '2-Line'
df_total.groupby('Line').sum()

,Model Boardings,Observed Spring 2023 (Pre-Lynnwood),Observed Fall 2024 (Post-Lynnwood)
Line,,,
1-Line,"88,728.74","79,522.00","84,250.00"


### 1-Line Lynnwood to Angle Lake

In [5]:
df_1_line

,Model Boardings,Observed Spring 2023 (Pre-Lynnwood),Observed Fall 2024 (Post-Lynnwood)
station_name,,,
Northgate Station,"9,403.29","10,384.00","4,150.00"
Roosevelt Station,"8,468.57","4,349.00","3,693.00"
University District Station,"9,009.79","7,563.00","6,004.00"
Husky Stadium,"3,919.56","6,134.00","5,122.00"
Capitol Hill,"8,174.24","7,667.00","8,207.00"
Westlake,"11,302.09","8,845.00","10,737.00"
Symphony,"6,422.60","4,324.00","4,783.00"
Pioneer Square,"5,205.49","2,400.00","3,008.00"
Int'l District,"1,839.46","4,399.00","5,372.00"


In [6]:
fig = px.line(df_1_line)
fig.update_layout(height=500, width=900, font=dict(size=11),
                  yaxis_tickformat = '.00f')
fig.show()

### 2-Line

In [7]:
df_2_line

,Model Boardings,Observed Spring 2023 (Pre-Lynnwood),Observed Fall 2024 (Post-Lynnwood)
station_name,,,


In [8]:
fig = px.line(df_2_line)
fig.update_layout(height=500, width=900, font=dict(size=11),
                  yaxis_tickformat = '.00f')
fig.show()